# Sayou Brain Quick Start

**Sayou Fabric**의 총괄 지휘자인 `sayou-brain`을 시연합니다.
이 노트북에서는 다음 두 가지 핵심 기능을 보여줍니다.

1.  **Plugin Injection:** 기본 기능 외에 `MarkdownSplitter` 같은 플러그인을 주입하여 파이프라인을 확장하는 방법.
2.  **Smart ETL:** 파일 타입(.md)을 인식하여 불필요한 파싱 단계를 건너뛰고 정제/분할로 직행하는 스마트 라우팅.

In [ ]:
import os
import json
import logging

# Brain
from sayou.brain.pipelines.standard import StandardPipeline

# Plugin to Inject
from sayou.chunking.plugins.markdown_splitter import MarkdownSplitter

# 로그 설정
logging.basicConfig(level=logging.INFO, format='%(message)s')

print("Import Successful!")

## 1. 파이프라인 구성 (Plugin Injection)

`MarkdownSplitter` 인스턴스를 생성하고, `StandardPipeline` 생성 시 `extra_splitters` 인자로 전달합니다.
Brain은 이를 받아 내부의 `ChunkingPipeline`에 등록합니다.

In [ ]:
# 1. 플러그인 준비
markdown_splitter = MarkdownSplitter()

# 2. Brain 생성 (플러그인 주입)
# extra_splitters 리스트에 플러그인들을 담아 전달합니다.
brain = StandardPipeline(extra_splitters=[markdown_splitter])

# 3. 설정 및 초기화
# (실제로는 config.yaml 등에서 로드)
config = {
    "chunking": {
        "chunk_size": 100,          # 테스트를 위해 작게 설정
        "structure_pattern": "\n\n" # 구조적 분할 패턴
    },
    "loader": {
        "mode": "w"                 # 덮어쓰기 모드
    }
}

brain.initialize(config=config)

## 2. 데이터 준비

테스트를 위해 Markdown 파일을 생성합니다.
이 파일은 `sayou-document` (바이너리 파싱) 단계를 건너뛰고, 바로 텍스트로 읽혀 `Refinery` -> `Chunking`으로 전달될 것입니다.

In [ ]:
INPUT_DIR = "examples"

md_content = """
# Sayou Architecture

Sayou Fabric is composed of modular libraries.

## 1. Core Modules
- Connector: Data Ingestion
- Brain: Orchestration

## 2. Advanced Modules
- Refinery: Cleaning
- Wrapper: Semantic Mapping
""".strip()

target_file = f"{INPUT_DIR}/arch.md"
with open(target_file, "w", encoding="utf-8") as f:
    f.write(md_content)

print(f"Created: {target_file}")

## 3. Ingest 실행

`strategies`에 `chunking: markdown`을 지정하여, 방금 주입한 플러그인이 작동하도록 합니다.

In [ ]:
DESTINATION = "brain_output.json"

result = brain.ingest(
    source=INPUT_DIR,
    destination=DESTINATION,
    strategies={
        "connector": "file",
        "chunking": "markdown",      # 주입한 플러그인 전략 사용
        "assembler": "graph",
        "loader": "file"
    }
)

print(f"\n✅ Stats: {result}")

## 4. 결과 검증

생성된 JSON 파일을 열어, Markdown 헤더 구조(`#`, `##`)가 `sayou:Topic` 노드로 잘 변환되었는지 확인합니다.

In [ ]:
if os.path.exists(DESTINATION):
    with open(DESTINATION, "r", encoding="utf-8") as f:
        data = json.load(f)
    
    print(f"Total Nodes: {len(data['nodes'])}")
    
    print("\n--- Node Samples ---")
    for node in data['nodes']:
        # Topic(헤더) 노드인지, TextFragment(본문) 노드인지 확인
        cls = node.get('node_class')
        text = node['attributes'].get('schema:text', '')
        print(f"[{cls}] {text}")